### Gibbs Sampler

We will estimate:

\begin{equation}
f(x,y) = k x^2 \exp(-xy^2 - y^2 + 2y - 4x)
\end{equation}

Using the conditional:

\begin{equation}
x|y \sim Gamma(3, y^2 +4)
\end{equation}

\begin{equation}
y|x \sim Normal(\frac{1}{1+x}, \frac{1}{2(1+x)})
\end{equation}

__sol__:

* _thin_ parameter means __Thinning__ which consists in picking separated points from the sample, at each k-th step

In [7]:
Rgibbs <- function(N,thin=500) {
    mat <- matrix(0,ncol=2,nrow=N)
    x <- 0
    y <- 0
    for (i in 1:N) { # samples
        for (j in 1:thin) { # thinning
            x <- rgamma(1,3,y*y+4)
            y <- rnorm(1,1/(x+1),1/sqrt(2*(x+1)))
        }
        mat[i,] <- c(x,y)
    }
    mat
}

In [10]:
mean(Rgibbs(100))

[1] 0.6492481

### Block Gibbs Sampler

In [30]:
n<-50 # number of observation
# simulate model matrix
x<-cbind(1, rnorm(n, 0, 1), rnorm(n, 5,10),rnorm(n, 100,10))
x

1,0.61312090,-7.88369373,122.93710
1,-0.27076716,5.08492354,83.85526
1,-0.43284572,1.51114167,67.91553
1,-0.06207595,8.34526483,94.40061
1,2.47939564,20.73992468,107.58426
1,0.44932399,2.21937383,97.81750
1,-1.57590059,5.14210836,95.30204
1,0.85091143,-2.00278140,96.53547
1,-1.48481718,-0.19239867,118.78096
1,-0.29764095,10.95758253,113.90466
1,1.12468608,9.98755964,107.21792


In [13]:
# true beta coefficients
tb<-c(1000, 50, -50, 10)

# true phi
tphi<-10000
I<-diag(1,n,n) # identity matrix used for covariance matrix

In [33]:
# simulate outcome for regression transpose
# %*% matrix multiplication 
y<-t(rmvnorm(1, x%*%tb, tphi*I))
length(y)

[1] 50

In [31]:
# simulate many outcomes...used later for asymptotic evaluations
y_list<-replicate(1000, t(rmvnorm(1, x%*%tb, tphi*I)),simplify = FALSE)
y_list[0]

list()

In [21]:
################################################################################
###### 1 - Run Blocked Gibbs Sampler
################################################################################

# function for blocked gibbs sampler
block_gibbs<-function(y, x, iter, burnin, trim){
  # initialize gibbs
  xprimex_inv<-solve(t(x)%*%x) # calculate once for repeated use in sampler
  phi<-numeric(iter) # shell for phi
  b<-matrix(nrow=iter, ncol = 4) # shell for betas
  phi[1]<-6 # initial phi value to start sampler
  
  # phi hyperparameters
  a<-.5
  g<-10000
  
  # gibbs sampling
  for(i in 2:iter ){
    b[i,]<-rmvnorm(n = 1, 
                   mean = ((xprimex_inv%*%t(x))%*%y), 
                   sigma = phi[i-1]*xprimex_inv )
    
    phi[i]<-rinvgamma(n = 1, 
                      shape = (n/2 + a), 
                      rate = .5*( t((y - x%*%t(t(b[i,])) ))%*%(y - x%*%t(t(b[i,])) ) ) + g)
  }
  
  # apply burnin and trimming  
  keep_draws<-seq(burnin,iter,trim)
  phi<-phi[keep_draws]
  b<-b[keep_draws,]
  
  # format and output
  joint_post<-data.frame(b=b,phi=phi)
  colnames(joint_post)[1:(ncol(x))]<-paste0('B',0:(ncol(x)-1) )
  
  joint_post_long<-gather(joint_post,keep_draws) %>%
    rename(param=keep_draws, draw=value) %>%
    mutate(iter=rep(keep_draws,ncol(joint_post)))
  
  return(joint_post_long)
}

In [35]:
iter = 500000
burnin = 100000
trim = 50

In [37]:
keep_draws<-seq(burnin,iter,trim)
keep_draws

[1] 100000 100050 100100 100150 100200 100250 100300 100350 100400 100450
  [11] 100500 100550 100600 100650 100700 100750 100800 100850 100900 100950
  [21] 101000 101050 101100 101150 101200 101250 101300 101350 101400 101450
  [31] 101500 101550 101600 101650 101700 101750 101800 101850 101900 101950
  [41] 102000 102050 102100 102150 102200 102250 102300 102350 102400 102450
  [51] 102500 102550 102600 102650 102700 102750 102800 102850 102900 102950
  [61] 103000 103050 103100 103150 103200 103250 103300 103350 103400 103450
  [71] 103500 103550 103600 103650 103700 103750 103800 103850 103900 103950
  [81] 104000 104050 104100 104150 104200 104250 104300 104350 104400 104450
  [91] 104500 104550 104600 104650 104700 104750 104800 104850 104900 104950
 [101] 105000 105050 105100 105150 105200 105250 105300 105350 105400 105450
 [111] 105500 105550 105600 105650 105700 105750 105800 105850 105900 105950
 [121] 106000 106050 106100 106150 106200 106250 106300 106350 106400 106450
 [131] 106500 106550 106600 106650 106700 106750 106800 106850 106900 106950
 [141] 107000 107050 107100 107150 107200 107250 107300 107350 107400 107450
 [151] 107500 107550 107600 107650 107700 107750 107800 107850 107900 107950
 [161] 108000 108050 108100 108150 108200 108250 108300 108350 108400 108450
 [171] 108500 108550 108600 108650 108700 108750 108800 108850 108900 108950
 [181] 109000 109050 109100 109150 109200 109250 109300 109350 109400 109450
 [191] 109500 109550 109600 109650 109700 109750 109800 109850 109900 109950
 [201] 110000 110050 110100 110150 110200 110250 110300 110350 110400 110450
 [211] 110500 110550 110600 110650 110700 110750 110800 110850 110900 110950
 [221] 111000 111050 111100 111150 111200 111250 111300 111350 111400 111450
 [231] 111500 111550 111600 111650 111700 111750 111800 111850 111900 111950
 [241] 112000 112050 112100 112150 112200 112250 112300 112350 112400 112450
 [251] 112500 112550 112600 112650 112700 112750 112800 112850 112900 112950
 [261] 113000 113050 113100 113150 113200 113250 113300 113350 113400 113450
 [271] 113500 113550 113600 113650 113700 113750 113800 113850 113900 113950
 [281] 114000 114050 114100 114150 114200 114250 114300 114350 114400 114450
 [291] 114500 114550 114600 114650 114700 114750 114800 114850 114900 114950
 [301] 115000 115050 115100 115150 115200 115250 115300 115350 115400 115450
 [311] 115500 115550 115600 115650 115700 115750 115800 115850 115900 115950
 [321] 116000 116050 116100 116150 116200 116250 116300 116350 116400 116450
 [331] 116500 116550 116600 116650 116700 116750 116800 116850 116900 116950
 [341] 117000 117050 117100 117150 117200 117250 117300 117350 117400 117450
 [351] 117500 117550 117600 117650 117700 117750 117800 117850 117900 117950
 [361] 118000 118050 118100 118150 118200 118250 118300 118350 118400 118450
 [371] 118500 118550 118600 118650 118700 118750 118800 118850 118900 118950
 [381] 119000 119050 119100 119150 119200 119250 119300 119350 119400 119450
 [391] 119500 119550 119600 119650 119700 119750 119800 119850 119900 119950
 [401] 120000 120050 120100 120150 120200 120250 120300 120350 120400 120450
 [411] 120500 120550 120600 120650 120700 120750 120800 120850 120900 120950
 [421] 121000 121050 121100 121150 121200 121250 121300 121350 121400 121450
 [431] 121500 121550 121600 121650 121700 121750 121800 121850 121900 121950
 [441] 122000 122050 122100 122150 122200 122250 122300 122350 122400 122450
 [451] 122500 122550 122600 122650 122700 122750 122800 122850 122900 122950
 [461] 123000 123050 123100 123150 123200 123250 123300 123350 123400 123450
 [471] 123500 123550 123600 123650 123700 123750 123800 123850 123900 123950
 [481] 124000 124050 124100 124150 124200 124250 124300 124350 124400 124450
 [491] 124500 124550 124600 124650 124700 124750 124800 124850 124900 124950
 [501] 125000 125050 125100 125150 125200 125250 125300 125350 125400 125450
 [511] 125500 125550 125600 125650 125700 125750 125800 125850 125900 125950

In [22]:
# run gibbs sampler with specified parameters
post_dist<-block_gibbs(y = y, x = x, iter = 500000, burnin = 100000, trim = 50)

ERROR: Error in rinvgamma(n = 1, shape = (n/2 + a), rate = 0.5 * (t((y - x %*% : could not find function "rinvgamma"


In [ ]:
################################################################################
###### 2 - Summarize and Visualize Posterior Distributions 
################################################################################

# calculate posterior summary statistics (stats not used in rest of code)
post_sum_stats<-post_dist %>%
  group_by(param) %>%
  summarise(median=median(draw),
            lwr=quantile(draw,.025),
            upr=quantile(draw,.975)) %>%
  mutate(true_vals=c(tb,tphi))

# merge on summary statistics
post_dist <- post_dist %>%
  left_join(post_sum_stats, by='param')

# plot MCMC Chains
ggplot(post_dist,aes(x=iter,y=draw)) +
  geom_line() +
  geom_hline(aes(yintercept=true_vals, col='red'), show.legend=FALSE)+
  facet_grid(param ~ .,scale='free_y',switch = 'y') +
  theme_bw() + 
  xlab('Gibbs Sample Iteration') + ylab('MCMC Chains') + 
  ggtitle('Gibbs Sampler MCMC Chains by Parameter')

# plot Posterior Distributions
ggplot(post_dist,aes(x=draw)) +
  geom_histogram(aes(x=draw),bins=50) +
  geom_vline(aes(xintercept = true_vals,col='red'), show.legend = FALSE) +
  facet_grid(. ~ param, scale='free_x',switch = 'y') +
  theme_bw() + 
  xlab('Posterior Distributions') + ylab('Count') + 
  ggtitle('Posterior Distributions of Parameters (true values in red)')

################################################################################
###### 3 - Assess Bias and Coverage
################################################################################

# run the estimation 1000 times to get 1000 posterior medians and CIs
bayes_res<-lapply(y_list, block_gibbs, x=x, iter=1000, burnin=100, trim=1)

calc_sumstats<-function(post_dist){
  post_sum_stats<-post_dist %>%
    group_by(param) %>%
    summarise(post_median=median(draw),
              lwr=quantile(draw,.025),
              upr=quantile(draw,.975)) %>%
    mutate(true_vals=c(tb,tphi))
  return(post_sum_stats)
}


all_sum_stats<-lapply(bayes_res, calc_sumstats)
all_sum_stats_stack<-bind_rows(all_sum_stats) %>%
  arrange(param) %>%
  rename(est=post_median)

eval_sum <- all_sum_stats_stack %>%
  mutate(covered=ifelse(true_vals<upr & true_vals>lwr,1,0)) %>%
  group_by(param) %>%
  summarise(est_var=var(est),
            est_mean=mean(est),
            bias=mean(est-true_vals),
            true_val=mean(true_vals),
            coverage=mean(covered)) %>%
  mutate(perc_bias=(bias/true_val)*100)

# format table
eval_sum<-eval_sum[,c(1,5,3,2,4,7,6)]
names(eval_sum)<-c('Parameter','True Value','Estimator Mean',
                   'Estimator Variance','Bias','Percent Bias (of truth)',
                   'Coverage of 95% CI')

print.xtable(xtable(eval_sum,
                    caption = 'Estimator Evaluation'),
             caption.placement = 'top', 
             include.rownames = F)